## License 

Copyright 2021--2022 Patrick Hall (jphall@gwu.edu)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

*DISCLAIMER*: This notebook is not legal or compliance advice.

# Model Evaluation Notebook

#### Imports and inits

In [1]:
import os              # for directory and file manipulation
import numpy as np     # for basic array manipulation
import pandas as pd    # for dataframe manipulation
import datetime        # for timestamp

# for model eval
from sklearn.metrics import accuracy_score, f1_score, log_loss, mean_squared_error, roc_auc_score

# global constants 
ROUND = 3              # generally, insane precision is not needed 
SEED = 12345           # seed for better reproducibility

# set global random seed for better reproducibility
np.random.seed(SEED)

#### Set basic metadata

In [2]:
y_name = 'high_priced'
scores_dir = 'data/scores'

#### Read in score files 

In [3]:
# init score frame with known test y values
scores_frame = pd.read_csv(scores_dir + os.sep +'key.csv', index_col='Unnamed: 0')

# create random folds in reproducible way
np.random.seed(SEED)
scores_frame['fold'] = np.random.choice(5, scores_frame.shape[0])

# read in each score file in the directory as a new column 
for file in os.listdir(scores_dir):
    if file != 'key.csv' and file.endswith('.csv'):
        scores_frame[file[:-4]] = pd.read_csv(scores_dir + os.sep + file)['phat']

# sanity check 
scores_frame

,high_priced,fold,group5_mxgb,group2_mxgb,group5_ebm,group6_mxgb,group8_mxgb,ph_mxgb,group7_piml_ebm,group2_ebm,...,group2_glm,group3_mxgb,group7_mxgb2,group6_piml_gaminet,group9_ebm,group6_glm,group9_glm,group8_glm,group3_glm,ph_ebm
0,0.0,2,0.059530,0.086361,0.081364,0.086361,0.078233,0.059522,0.074151,0.081364,...,0.142090,0.059522,0.080985,0.034641,0.081364,0.142090,0.142090,0.142090,0.142090,0.082841
1,0.0,1,0.036380,0.033920,0.026177,0.033920,0.022467,0.036210,0.031165,0.026177,...,0.081674,0.036210,0.033100,0.032192,0.026177,0.081674,0.081674,0.081674,0.081674,0.027079
2,1.0,4,0.180033,0.183323,0.184662,0.183323,0.179896,0.180734,0.182987,0.184662,...,0.125823,0.180734,0.177759,0.257928,0.184662,0.125823,0.125823,0.125823,0.125823,0.190718
3,0.0,1,0.027747,0.030934,0.030049,0.030934,0.010161,0.027677,0.018368,0.030049,...,0.006973,0.027677,0.023707,0.000700,0.030049,0.006973,0.006973,0.006973,0.006973,0.031069
4,1.0,2,0.178084,0.178491,0.205948,0.178491,0.214328,0.177813,0.188477,0.205948,...,0.130426,0.177813,0.210616,0.124897,0.205948,0.130426,0.130426,0.130426,0.130426,0.210361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19826,0.0,3,0.275183,0.255826,0.227771,0.255826,0.236099,0.274767,0.203201,0.227771,...,0.160032,0.274767,0.257779,0.218879,0.227771,0.160032,0.160032,0.160032,0.160032,0.231624
19827,0.0,1,0.182247,0.176984,0.253325,0.176984,0.255476,0.182039,0.207900,0.253325,...,0.123836,0.182039,0.252247,0.145454,0.253325,0.123836,0.123836,0.123836,0.123836,0.254823
19828,1.0,3,0.212705,0.236894,0.225811,0.236894,0.224857,0.212740,0.215871,0.225811,...,0.169604,0.212740,0.204329,0.062602,0.225811,0.169604,0.169604,0.169604,0.169604,0.220400
19829,0.0,1,0.001348,0.001113,0.001488,0.001113,0.001779,0.001323,0.007910,0.001488,...,0.002538,0.001323,0.000406,0.000042,0.001488,0.002538,0.002538,0.002538,0.002538,0.000993


#### Utility function for max. accuracy

In [4]:
def max_acc(y, phat, res=0.01): 

    """ Utility function for finding max. accuracy at some cutoff. 
    
        :param y: Known y values.
        :param phat: Model scores.
        :param res: Resolution over which to search for max. accuracy, default 0.01.
        :return: Max. accuracy for model scores.
    
    """
    
    # init frame to store acc at different cutoffs
    acc_frame = pd.DataFrame(columns=['cut', 'acc'])
    
    # copy known y and score values into a temporary frame
    temp_df = pd.concat([y, phat], axis=1)
    
    # find accuracy at different cutoffs and store in acc_frame
    for cut in np.arange(0, 1 + res, res):
        temp_df['decision'] = np.where(temp_df.iloc[:, 1] > cut, 1, 0)
        acc = accuracy_score(temp_df.iloc[:, 0], temp_df['decision'])
        acc_frame = acc_frame.append({'cut': cut,
                                      'acc': acc},
                                     ignore_index=True)

    # find max accurcay across all cutoffs
    max_acc = acc_frame['acc'].max()
    
    # house keeping
    del acc_frame, temp_df
    
    return max_acc

####  Utility function for max. F1

In [5]:
def max_f1(y, phat, res=0.01): 
    
    """ Utility function for finding max. F1 at some cutoff. 
    
        :param y: Known y values.
        :param phat: Model scores.
        :param res: Resolution over which to search for max. F1, default 0.01.
        :return: Max. F1 for model scores.
    
    """
    
    # init frame to store f1 at different cutoffs
    f1_frame = pd.DataFrame(columns=['cut', 'f1'])
    
    # copy known y and score values into a temporary frame
    temp_df = pd.concat([y, phat], axis=1)
    
    # find f1 at different cutoffs and store in acc_frame
    for cut in np.arange(0, 1 + res, res):
        temp_df['decision'] = np.where(temp_df.iloc[:, 1] > cut, 1, 0)
        f1 = f1_score(temp_df.iloc[:, 0], temp_df['decision'])
        f1_frame = f1_frame.append({'cut': cut,
                                    'f1': f1},
                                    ignore_index=True)
        
    # find max f1 across all cutoffs
    max_f1 = f1_frame['f1'].max()
    
     # house keeping
    del f1_frame, temp_df
    
    return max_f1

#### Rank all submitted scores 

In [6]:
eval_frame = pd.DataFrame() # init frame to hold score ranking
metric_list = ['acc', 'auc', 'f1', 'logloss', 'mse'] # metric to use for evaluation

# create eval frame row-by-row
for fold in sorted(scores_frame['fold'].unique()): # loop through folds 
    for metric_name in metric_list: # loop through metrics
        
        # init row dict to hold each rows values
        row_dict = {'fold': fold,
                    'metric': metric_name}
        
        # cache known y values for fold
        fold_y = scores_frame.loc[scores_frame['fold'] == fold, y_name]
        
        for col_name in scores_frame.columns[2:]:
            
            # cache fold scores
            fold_scores = scores_frame.loc[scores_frame['fold'] == fold, col_name]
            
            # calculate evaluation metric for fold
            # with reasonable precision 
            
            if metric_name == 'acc':
                row_dict[col_name] = np.round(max_acc(fold_y, fold_scores), ROUND)
                
            if metric_name == 'auc':
                row_dict[col_name] = np.round(roc_auc_score(fold_y, fold_scores), ROUND)
                
            if metric_name == 'f1':
                row_dict[col_name] = np.round(max_f1(fold_y, fold_scores), ROUND) 
                
            if metric_name == 'logloss':
                row_dict[col_name] = np.round(log_loss(fold_y, fold_scores), ROUND)
                
            if metric_name == 'mse':
                row_dict[col_name] = np.round(mean_squared_error(fold_y, fold_scores), ROUND)
        
        # append row values to eval_frame
        eval_frame = eval_frame.append(row_dict, ignore_index=True)

# init a temporary frame to hold rank information
rank_names = [name + '_rank' for name in eval_frame.columns if name not in ['fold', 'metric']]
rank_frame = pd.DataFrame(columns=rank_names)        

# set columns to necessary order
eval_frame = eval_frame[['fold', 'metric'] + [name for name in sorted(eval_frame.columns) if name not in ['fold', 'metric']]]

# determine score ranks row-by-row
for i in range(0, eval_frame.shape[0]):
        
        # get ranks for row based on metric
        metric_name = eval_frame.loc[i, 'metric']
        if metric_name in ['logloss', 'mse']:
            ranks = eval_frame.iloc[i, 2:].rank().values
        else:
            ranks = eval_frame.iloc[i, 2:].rank(ascending=False).values
        
        # create single-row frame and append to rank_frame
        row_frame = pd.DataFrame(ranks.reshape(1, ranks.shape[0]), columns=rank_names)
        rank_frame = rank_frame.append(row_frame, ignore_index=True)
        
        # house keeping
        del row_frame

# merge ranks onto eval_frame
eval_frame = pd.concat([eval_frame, rank_frame], axis=1)

# house keeping
del rank_frame
        
eval_frame

,fold,metric,group1_ebm,group1_glm,group1_mxgb,group2_ebm,group2_glm,group2_mxgb,group3_ebm,group3_glm,...,group7_piml_reludnn_rank,group8_ebm_rank,group8_glm_rank,group8_mxgb_rank,group9_ebm_rank,group9_glm_rank,group9_mxgb_rank,ph_ebm_rank,ph_glm_rank,ph_mxgb_rank
0,0.0,acc,0.901,0.900,0.902,0.901,0.900,0.902,0.901,0.900,...,16.0,16.0,28.5,16.0,16.0,28.5,5.0,16.0,28.5,5.0
1,0.0,auc,0.839,0.775,0.812,0.839,0.775,0.813,0.839,0.775,...,14.0,6.0,30.0,1.0,6.0,30.0,18.0,6.0,30.0,18.0
2,0.0,f1,0.404,0.335,0.376,0.404,0.335,0.374,0.404,0.335,...,20.0,7.0,30.0,2.0,7.0,30.0,16.5,1.0,30.0,16.5
3,0.0,logloss,0.251,0.291,0.264,0.251,0.291,0.263,0.251,0.291,...,14.0,6.0,30.0,1.0,6.0,30.0,18.5,6.0,30.0,18.5
4,0.0,mse,0.077,0.084,0.078,0.077,0.084,0.078,0.077,0.084,...,18.0,7.0,30.0,7.0,7.0,30.0,18.0,7.0,30.0,18.0
5,1.0,acc,0.906,0.906,0.906,0.906,0.906,0.906,0.906,0.906,...,18.5,18.5,18.5,1.5,18.5,18.5,18.5,18.5,18.5,18.5
6,1.0,auc,0.829,0.757,0.793,0.829,0.757,0.792,0.829,0.757,...,15.0,5.5,30.0,1.0,5.5,30.0,20.5,10.5,30.0,20.5
7,1.0,f1,0.371,0.302,0.339,0.371,0.302,0.342,0.371,0.302,...,14.0,6.0,30.0,1.0,6.0,30.0,21.0,10.5,30.0,21.0
8,1.0,logloss,0.246,0.281,0.263,0.246,0.281,0.264,0.246,0.281,...,14.0,6.0,30.0,1.0,6.0,30.0,20.5,6.0,30.0,20.5
9,1.0,mse,0.074,0.080,0.078,0.074,0.080,0.078,0.074,0.080,...,14.0,5.5,29.0,5.5,5.5,29.0,19.5,5.5,29.0,19.5


#### Save `eval_frame` as CSV

In [7]:
eval_frame.to_csv('model_eval_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '.csv'), 
                  index=False)

#### Display simple ranked score list 

In [8]:
eval_frame[[name for name in eval_frame.columns if name.endswith('rank')]].mean().sort_values()

group8_mxgb_rank             3.74
ph_ebm_rank                  7.84
group1_ebm_rank              8.04
group2_ebm_rank              8.04
group9_ebm_rank              8.04
group3_ebm_rank              8.04
group8_ebm_rank              8.04
group5_ebm_rank              8.04
group7_ebm_rank              8.04
group6_ebm_rank              8.28
group7_mxgb2_rank            9.24
group7_piml_ebm_rank        12.16
group6_piml_ebm_rank        12.16
group7_piml_reludnn_rank    15.76
group7_piml_gaminet_rank    18.00
group9_mxgb_rank            18.34
ph_mxgb_rank                18.34
group3_mxgb_rank            18.34
group5_mxgb_rank            18.36
group6_mxgb_rank            19.46
group2_mxgb_rank            19.46
group1_mxgb_rank            19.66
group7_mxgb_rank            19.66
group6_piml_reludnn_rank    19.72
group6_piml_gaminet_rank    27.66
ph_glm_rank                 28.06
group8_glm_rank             28.06
group6_glm_rank             28.06
group2_glm_rank             28.06
group9_glm_ran